In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir("/content/gdrive/My Drive/BTP-SEM-8/VideoAnomalyDetection/training")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
%tensorflow_version 2.x
from tensorflow.python.keras import Sequential
from tensorflow.python.keras.layers import Dense, Dropout, Activation,LSTM,Reshape
from tensorflow.python.keras.regularizers import l2
from tensorflow.keras.optimizers import SGD,Adam, Adagrad
from scipy.io import loadmat, savemat
from tensorflow.python.keras.models import model_from_json
import theano.tensor as T
import theano
import csv
import collections
import time
import csv
import os
from os import listdir
import skimage.transform
from skimage import color
from os.path import isfile, join
import numpy as np
import numpy
from datetime import datetime
from os.path import basename
import glob
import theano.sandbox
import pickle
import tensorflow as tf

In [0]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [0]:
print("Create Model")
model = Sequential()
model.add(Dense(512, input_dim=4096,kernel_initializer='glorot_normal',kernel_regularizer=l2(0.001),activation='relu'))
model.add(Dropout(0.6))
model.add(Dense(32,kernel_initializer='glorot_normal',kernel_regularizer=l2(0.001)))
model.add(Dropout(0.6))
model.add(Dense(1,kernel_initializer='glorot_normal',kernel_regularizer=l2(0.001),activation='sigmoid'))

Create Model


In [0]:
def load_model(json_path): # Function to load the model
    model = model_from_json(open(json_path).read())
    return model

def load_weights(model, weight_path): # Function to load the model weights
    dict2 = loadmat(weight_path)
    dict = conv_dict(dict2)
    i = 0
    for layer in model.layers:
        weights = dict[str(i)]
        layer.set_weights(weights)
        i += 1
    return model

def conv_dict(dict2):
    i = 0
    dict = {}
    for i in range(len(dict2)):
        if str(i) in dict2:
            if dict2[str(i)].shape == (0, 0):
                dict[str(i)] = dict2[str(i)]
            else:
                weights = dict2[str(i)][0]
                weights2 = []
                for weight in weights:
                    if weight.shape in [(1, x) for x in range(0, 5000)]:
                        weights2.append(weight[0])
                    else:
                        weights2.append(weight)
                dict[str(i)] = weights2
    return dict

def save_model(savemodel, json_path, weight_path): # Function to save the model
    json_string = model.to_json()
    open(json_path, 'w').write(json_string)
    dict = {}
    i = 0
    for layer in model.layers:
        weights = layer.get_weights()
        my_list = np.zeros(len(weights), dtype=np.object)
        my_list[:] = weights
        dict[str(i)] = my_list
        i += 1
    savemat(weight_path, dict)

In [0]:
# Load Training Dataset

def load_dataset_Train_batch(abnormal_videos_matrix, normal_videos_matrix):
#    print("Loading training batch")

    batchsize=60       # Each batch contain 60 videos.
    n_exp=int(batchsize/2)  # Number of abnormal and normal videos

    Num_abnormal = 810  # Total number of abnormal videos in Training Dataset.
    Num_Normal = 800    # Total number of Normal videos in Training Dataset.

    # We assume the features of abnormal videos and normal videos are located in two different folders.
    Abnor_list_iter = np.random.permutation(Num_abnormal)
    Abnor_list_iter = Abnor_list_iter[Num_abnormal-n_exp:] # Indexes for randomly selected Abnormal Videos
    Norm_list_iter = np.random.permutation(Num_Normal)
    Norm_list_iter = Norm_list_iter[Num_Normal-n_exp:]     # Indexes for randomly selected Normal Videos


    AllFeatures = []  # To store Cad3D features of a batch
    #print("Loading Abnormal videos Features...")

    Video_count=-1
    for iv in Abnor_list_iter:
        Video_count=Video_count+1
        words = abnormal_videos_matrix[iv]
        num_feat = int(len(words) / 4096)
        # Number of features per video to be loaded. In our case num_feat=32, as we divide the video into 32 segments. Note that
        # we have already computed C3D features for the whole video and divide the video features into 32 segments. Please see Save_C3DFeatures_32Segments.m as well

        count = -1
        VideoFeatues = []
        for feat in range(0, num_feat):
            feat_row1 = np.float32(words[feat * 4096:feat * 4096 + 4096])
            count = count + 1
            if count == 0:
                VideoFeatues = feat_row1
            if count > 0:
                VideoFeatues = np.vstack((VideoFeatues, feat_row1))

        if Video_count == 0:
            AllFeatures = VideoFeatues
        if Video_count > 0:
            AllFeatures = np.vstack((AllFeatures, VideoFeatues))
        # print(" Abnormal Features  loaded")

        
        
    #print("Loading Normal videos...")

    for iv in Norm_list_iter:
        words = normal_videos_matrix[iv]
        feat_row1 = np.array([])
        num_feat = int(len(words) /4096)   # Number of features to be loaded. In our case num_feat=32, as we divide the video into 32 segments.

        count = -1
        VideoFeatues = []
        for feat in range(0, num_feat):


            feat_row1 = np.float32(words[feat * 4096:feat * 4096 + 4096])
            count = count + 1
            if count == 0:
                VideoFeatues = feat_row1
            if count > 0:
                VideoFeatues = np.vstack((VideoFeatues, feat_row1))
            feat_row1 = []
        AllFeatures = np.vstack((AllFeatures, VideoFeatues))

    #print("Features  loaded")


    AllLabels = np.zeros(32*batchsize, dtype='uint8')
    th_loop1=n_exp*32
    th_loop2=n_exp*32-1



    for iv in range(0, 32*batchsize):
            if iv< th_loop1:
                AllLabels[iv] = int(1)  # All instances of abnormal videos are labeled 1.  This will be used in custom_objective to keep track of normal and abnormal videos indexes.
            if iv > th_loop2:
                AllLabels[iv] = int(0)   # All instances of Normal videos are labeled 0. This will be used in custom_objective to keep track of normal and abnormal videos indexes.
           # print("ALLabels  loaded")

    return  AllFeatures,AllLabels


In [0]:
# Load Testing Dataset

def load_dataset_Test_batch(abnormal_videos_matrix, normal_videos_matrix):
#    print("Loading training batch")

    #batchsize=60       # Each batch contain 60 videos.
    #n_exp=int(batchsize/2)  # Number of abnormal and normal videos

    #Num_abnormal = 810  # Total number of abnormal videos in Training Dataset.
    #Num_Normal = 800    # Total number of Normal videos in Training Dataset.


    # We assume the features of abnormal videos and normal videos are located in two different folders.
    #Abnor_list_iter = np.random.permutation(Num_abnormal)
    #Abnor_list_iter = Abnor_list_iter[Num_abnormal-n_exp:] # Indexes for randomly selected Abnormal Videos
    #Norm_list_iter = np.random.permutation(Num_Normal)
    #Norm_list_iter = Norm_list_iter[Num_Normal-n_exp:]     # Indexes for randomly selected Normal Videos


    AllFeatures = []  # To store Cad3D features of all testing videos
    #print("Loading Abnormal videos Features...")

    Video_count=-1
    for iv in range(len(abnormal_videos_matrix)):
        Video_count=Video_count+1
        words = abnormal_videos_matrix[iv]
        num_feat = int(len(words) / 4096)
        # Number of features per video to be loaded. In our case num_feat=32, as we divide the video into 32 segments. Note that
        # we have already computed C3D features for the whole video and divide the video features into 32 segments. Please see Save_C3DFeatures_32Segments.m as well

        count = -1
        VideoFeatues = []
        for feat in range(0, num_feat):
            feat_row1 = np.float32(words[feat * 4096:feat * 4096 + 4096])
            count = count + 1
            if count == 0:
                VideoFeatues = feat_row1
            if count > 0:
                VideoFeatues = np.vstack((VideoFeatues, feat_row1))

        if Video_count == 0:
            AllFeatures = VideoFeatues
        if Video_count > 0:
            AllFeatures = np.vstack((AllFeatures, VideoFeatues))
        # print(" Abnormal Features  loaded")

        
        
    #print("Loading Normal videos...")

    for iv in range(len(normal_videos_matrix)):
        words = normal_videos_matrix[iv]
        feat_row1 = np.array([])
        num_feat = int(len(words) /4096)   # Number of features to be loaded. In our case num_feat=32, as we divide the video into 32 segments.

        count = -1
        VideoFeatues = []
        for feat in range(0, num_feat):


            feat_row1 = np.float32(words[feat * 4096:feat * 4096 + 4096])
            count = count + 1
            if count == 0:
                VideoFeatues = feat_row1
            if count > 0:
                VideoFeatues = np.vstack((VideoFeatues, feat_row1))
            feat_row1 = []
        AllFeatures = np.vstack((AllFeatures, VideoFeatues))

    #print("Features  loaded")


    AllLabels = np.zeros(len(abnormal_videos_matrix) + len(normal_videos_matrix), dtype='uint8')
    th_loop1=len(abnormal_videos_matrix)
    th_loop2=len(abnormal_videos_matrix)-1



    for iv in range(0, len(abnormal_videos_matrix) + len(normal_videos_matrix)):
            if iv< th_loop1:
                AllLabels[iv] = int(1)  # All instances of abnormal videos are labeled 1.  This will be used in custom_objective to keep track of normal and abnormal videos indexes.
            if iv > th_loop2:
                AllLabels[iv] = int(0)   # All instances of Normal videos are labeled 0. This will be used in custom_objective to keep track of normal and abnormal videos indexes.
           # print("ALLabels  loaded")

    return  AllFeatures,AllLabels


In [0]:
#Tensorflow
def custom_objective(y_true, y_pred):
    'Custom Objective function'

    #print(type(y_true))
    y_true = tf.keras.backend.flatten(y_true)
    y_pred = tf.keras.backend.flatten(y_pred)

    n_seg = 32  # Because we have 32 segments per video.
    nvid = 60
    n_exp = int(nvid/2)
    Num_d=32*nvid


    sub_max = tf.ones_like(y_pred) # sub_max represents the highest scoring instants in bags (videos).
    sub_sum_labels = tf.ones_like(y_true) # It is used to sum the labels in order to distinguish between normal and abnormal videos.
    sub_sum_l1=tf.ones_like(y_true, dtype='float32')  # For holding the concatenation of summation of scores in the bag.
    sub_l2 = tf.ones_like(y_true, dtype='float32') # For holding the concatenation of L2 of score in the bag.

    for ii in range(0, nvid, 1):
        # For Labels
        mm = y_true[ii * n_seg:ii * n_seg + n_seg]
        sub_sum_labels = tf.concat([sub_sum_labels, tf.stack([tf.math.reduce_sum(mm)])],0) # Just to keep track of abnormal and normal vidoes
        # For Features scores
        Feat_Score = y_pred[ii * n_seg:ii * n_seg + n_seg]
        sub_max = tf.concat([sub_max, tf.stack([tf.math.reduce_max(Feat_Score)])],0)         # Keep the maximum score of scores of all instances in a Bag (video)
        sub_sum_l1 = tf.concat([sub_sum_l1, tf.stack([tf.math.reduce_sum(Feat_Score)])],0)   # Keep the sum of scores of all instances in a Bag (video)

        z1 = tf.ones_like(Feat_Score)
        z2 = tf.concat([z1, Feat_Score],0)
        z3 = tf.concat([Feat_Score, z1],0)
        z_22 = z2[31:]
        z_44 = z3[:33]
        z = z_22 - z_44
        z = z[1:32]
        z = tf.math.reduce_sum(tf.math.square(z))
        sub_l2 = tf.concat([sub_l2, tf.stack([z])],0)


    # sub_max[Num_d:] means include all elements after Num_d.
    # AllLabels =[2 , 4, 3 ,9 ,6 ,12,7 ,18 ,9 ,14]
    # z=x[4:]
    #[  6.  12.   7.  18.   9.  14.]

    sub_score = sub_max[Num_d:]  # We need this step since we have used tf.ones_like
    F_labels = sub_sum_labels[Num_d:] # We need this step since we have used tf.ones_like
    #  F_labels contains integer 32 for normal video and 0 for abnormal videos. This because of labeling done at the end of "load_dataset_Train_batch"



    # AllLabels =[2 , 4, 3 ,9 ,6 ,12,7 ,18 ,9 ,14]
    # z=x[:4]
    # [ 2 4 3 9]... This shows 0 to 3 elements

    sub_sum_l1 = sub_sum_l1[Num_d:] # We need this step since we have used tf.ones_like
    sub_sum_l1 = sub_sum_l1[:n_exp]
    sub_l2 = sub_l2[Num_d:]         # We need this step since we have used tf.ones_like
    sub_l2 = sub_l2[:n_exp]


    indx_nor = tf.squeeze(tf.where(tf.equal(F_labels,0)))
    indx_abn = tf.squeeze(tf.where(tf.equal(F_labels,32))) # Index of abormal videos: Since we labeled 1 for each of 32 segments of normal videos F_labels=32 for normal video


    #print("Index_nor:", indx_nor.shape)
    #print("sub_score:", sub_score.shape)

    n_Nor=n_exp

    #Sub_Nor = sub_score[indx_nor] # Maximum Score for each of abnormal video
    #Sub_Abn = sub_score[indx_abn] # Maximum Score for each of normal video
    Sub_Nor = tf.gather(sub_score, indx_nor) 
    #print("Sub_Nor:", Sub_Nor.shape)
    Sub_Abn = tf.gather(sub_score, indx_abn) 

    z = tf.ones_like(y_true, dtype='float32')
    for ii in range(0, n_Nor, 1):
        sub_z = tf.math.maximum(1 - Sub_Abn + Sub_Nor[ii], 0)
        z = tf.concat([z, tf.stack([tf.math.reduce_sum(sub_z)])],0)

    z = z[Num_d:]  # We need this step since we have used tf.ones_like
    z = tf.math.reduce_mean(z, axis=-1) +  0.00008*tf.math.reduce_sum(sub_sum_l1) + 0.00008*tf.math.reduce_sum(sub_l2)  # Final Loss f
    #print("Loss Z:", z)
    return z

In [0]:
adagrad=Adagrad(lr=0.01, epsilon=1e-08)

start_itr = 20001
end_itr = 40000


print("Starting training...")
output_dir='trained_models/'
# Output_dir is the directory where you want to save trained weights
weights_path = output_dir + 'weightsAnomalyL1L2_' + str(start_itr-1) + '.mat'
# weights.mat are the model weights that you will get after (or during) that training
model_path = output_dir + 'model.json'

if start_itr != 0:
  model = load_model(model_path)
  model = load_weights(model, weights_path)
model.compile(loss=custom_objective, optimizer=adagrad)

if not os.path.exists(output_dir):
       os.makedirs(output_dir)

loss_graph =[]
total_iterations = 0
batchsize=60
time_before = datetime.now()

abnormal_videos_matrix = pickle.load(open("../c3d_predictions/Anomaly/Train_features/train_features.pkl", "rb")) #dim: [n, 131072]
normal_videos_matrix = pickle.load(open("../c3d_predictions/Normal/Train_features/train_features.pkl", "rb")) #dim: [n, 131072]


for it_num in range(start_itr, end_itr):
    inputs, targets=load_dataset_Train_batch(abnormal_videos_matrix, normal_videos_matrix)  # Load normal and abnormal video C3D features
    #print("Inputs shape:", inputs.shape)
    #print("Targets shape:", targets.shape)
    batch_loss = model.train_on_batch(inputs, targets)
    loss_graph = np.hstack((loss_graph, batch_loss))
    total_iterations += 1
    if total_iterations % 20 == 1:
        print ("These iteration=" + str(total_iterations) + ") took: " + str(datetime.now() - time_before) + ", with loss of " + str(batch_loss))
        iteration_path = output_dir + 'Iterations_graph_' + str(it_num) + '.mat'
        #savemat(iteration_path, dict(loss_graph=loss_graph))
    if total_iterations % 1000 == 0:  # Save the model at every 1000th iterations.
        weights_path = output_dir + 'weightsAnomalyL1L2_' + str(it_num) + '.mat'
        save_model(model, model_path, weights_path)

weights_path = output_dir + 'weightsAnomalyL1L2_' + str(end_itr) + '.mat'
save_model(model, model_path, weights_path)

Starting training...
These iteration=1) took: 0:00:36.728310, with loss of 1.4481797
These iteration=21) took: 0:00:44.316699, with loss of 1.4583732
These iteration=41) took: 0:00:52.219398, with loss of 2.515462
These iteration=61) took: 0:01:00.209531, with loss of 1.5231242
These iteration=81) took: 0:01:07.918564, with loss of 1.4799616
These iteration=101) took: 0:01:15.523063, with loss of 2.4449592
These iteration=121) took: 0:01:23.145024, with loss of 1.4486849
These iteration=141) took: 0:01:31.168183, with loss of 1.4593409
These iteration=161) took: 0:01:39.168018, with loss of 5.457041
These iteration=181) took: 0:01:47.020718, with loss of 3.4846277
These iteration=201) took: 0:01:55.280215, with loss of 0.46397537
These iteration=221) took: 0:02:03.336507, with loss of 3.4612932
These iteration=241) took: 0:02:11.180312, with loss of 1.4706018
These iteration=261) took: 0:02:19.093562, with loss of 1.4366779
These iteration=281) took: 0:02:26.995679, with loss of 2.4640

In [0]:
abnormal_videos_matrix = pickle.load(open("../c3d_predictions/Anomaly/Test_features/test_features.pkl", "rb")) #dim: [n, 131072]
normal_videos_matrix = pickle.load(open("../c3d_predictions/Normal/Test_features/test_features.pkl", "rb")) #dim: [n, 131072]
inputs, targets = load_dataset_Test_batch(abnormal_videos_matrix, normal_videos_matrix)

In [0]:
print("Num abnormal:", len(abnormal_videos_matrix))
print("Num normal:", len(normal_videos_matrix))
print("Length of inputs and targets: ", len(inputs), len(targets))

Num abnormal: 140
Num normal: 150
Length of inputs and targets:  9280 9280


In [0]:
num_iters = 20000
output_dir='trained_models/'
# Output_dir is the directory where you want to save trained weights
# weights_path = output_dir + 'weightsAnomalyL1L2_' + str(num_iters) + '.mat'

#running previous trained model (with opposite labels)
weights_path = output_dir + 'weights_L1L2_ptirupat' + '.mat'

# weights_path = output_dir + 'weights_L1L2_pitrupat.mat'
# weights.mat are the model weights that you will get after (or during) that training
model_path = output_dir + 'model.json'


model = load_model(model_path)
model = load_weights(model, weights_path)

predictions = model.predict(inputs)

In [0]:
predictions[5:9]

array([[1.9302897e-09],
       [8.8198138e-10],
       [4.7204646e-10],
       [4.2839626e-10]], dtype=float32)

In [0]:
#running this for predictions with opposite labeling

predictions = [1-pred for pred in predictions]
predictions = np.array(predictions)

In [0]:
predictions[5:9]

[array([1.], dtype=float32),
 array([1.], dtype=float32),
 array([1.], dtype=float32),
 array([0.99999994], dtype=float32)]

In [0]:
predictions.shape

(9280, 1)

In [0]:
ans=[]
k=int(9280/32)
for i in range(k):
    #predictions=(predictions>0.0000001)*1
    val=sum(predictions[i*32:(i*32+32)]>0.6)
    if(val<32):
        ans.append(0) #abnormal
    else:
        ans.append(1) #normal

In [0]:
sum(targets==ans)/len(targets)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  """Entry point for launching an IPython kernel.


TypeError: ignored

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp
from sklearn.metrics import roc_auc_score

In [0]:
targets = np.ones((len(abnormal_videos_matrix)*32))
targets = np.concatenate((targets,np.zeros((len(normal_videos_matrix)*32))))
fpr, tpr, thresholds = roc_curve(targets, predictions)
roc_auc = auc(fpr, tpr)
roc_auc

In [0]:
plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()